In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.utilities import read_gifti
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define metadata.
subj = 'sub-01'
task = 'visualcontrol'
hemis = ['L', 'R']

## Section 1: Filter Data

In [ ]:
from nilearn.signal import clean

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Filter parameters.
high_pass = 1 / 100
t_r = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
for hemi in hemis:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load data.
    data = read_gifti('data/%s_task-%s_bold_space-fsaverage6.%s.func.gii' %(subj,task,hemi))

    ## Identify nonzero vertices.
    mask = np.abs(data).sum(axis=0).astype(bool)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Filter and convert to PSC.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Compute mean signal.
    mu = data[:,mask].mean(axis=0)

    ## Apply highpass data.
    data[:,mask] = clean(data[:,mask], detrend=True, standardize=False, 
                         high_pass=high_pass, t_r=t_r)

    ## Convert to percent signal change.
    data[:,mask] = data[:,mask] / mu * 100

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    np.savez_compressed('data/data-%s.npz' %hemi, data=data, mask=mask)
            
print('Done.')

## Section 2: Generate Task Regressors

In [ ]:
from scripts.spm_hrf import spm_hrf

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Task metadata.
n_acq = 250
tr = 1

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Generate task regressors.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Generate HRF.
hrf = spm_hrf(1)

## Generate boxcars.
boxcars = np.zeros(250)
for onset in np.arange(10,250,40):
    boxcars[onset:onset+20] = 1
    
## Generate idealized regressor.
bold = np.convolve(boxcars, hrf)[:boxcars.size]
bold /= bold.max()

## Generate design matrix.
X = np.vstack([bold, np.ones_like(bold)]).T

## Section 3: Mass Univariate

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
for hemi in hemis:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load data.
    npz = np.load('data/data-%s.npz' %hemi)

    ## Extract data.
    data = npz['data']
    mask = npz['mask']
    
    ## Mask data.
    Y = data[:,mask]
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Perform mass univariate.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ## Smoothing will be performed afterwards.
    
    ## Perform OLS.
    W, _, _, _ = np.linalg.lstsq(X, Y)
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Convert to Nifti image.
    overlay = np.zeros_like(mask, dtype=float)
    overlay[mask] = W[0]
    overlay = nib.Nifti1Image(overlay.reshape(-1,1,1,1), np.identity(4))
    
    ## Save.
    nib.save(overlay, 'data/data_ols_sm03-%s.nii.gz' %hemi)
        
print('Done.')

## Section 4: IAR

In [ ]:
from mne import read_surface
from scipy.sparse import coo_matrix
from scripts.matnormal import MatrixNormalCAR
from scripts.utilities import tris_to_adj

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for hemi in hemis[:1]:

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load surface / label data.
    _, tris = read_surface('data/%s.white' %('lh' if hemi=='L' else 'rh'))

    ## Load data.
    npz = np.load('data/data-%s.npz' %hemi)

    ## Extract data.
    data = npz['data']
    mask = npz['mask']
    
    ## Mask data.
    Y = data[:,mask]

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Remove triangles with vertices in the medial wall.
    vertices, = np.where(np.invert(mask))
    ix = np.any(np.apply_along_axis(np.in1d, 0, tris, vertices), axis=-1)
    tris = tris[np.invert(ix)]

    ## Make adjacency matrix.
    A = tris_to_adj(tris, remap_vertices=True, verbose=False).tocsc()

    ## Compute degree.
    D = np.asarray(A.sum(axis=0)).squeeze()
    D = coo_matrix((D, (np.arange(D.size), np.arange(D.size))), shape=[D.size,D.size]).tocsc()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Fit IAR model.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    mn = MatrixNormalCAR()
    W = mn.fit(Y, X, D, A)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Convert to Nifti image.
    overlay = np.zeros_like(mask, dtype=float)
    overlay[mask] = W.reshape(-1,2)[:,0]
    overlay = nib.Nifti1Image(overlay.reshape(-1,1,1,1), np.identity(4))

    ## Save.
    nib.save(overlay, 'data/data_iar-%s.nii.gz' %hemi)
    
print('Done.')